Chapter 4. Relabeling
=======

Recall that **Variable**s in **Prove-It** are arbitrary labels that are interchangeable. In `tutorial_01_core_expr`, we demonstrated using the `relabeled` method of **Expression** to transform an expression to one with **Variable**s swapped for other **Variable**s.  This was, however, only a transformation of the expression with no proof implication.  Here we will discuss the *relabeling* derivation step in which we can derive a new **Judgment** from an existing **Judgment** by swapping **Variable**s for other **Variable**s.

## Attempting to relabel a free Variable of an assumption

Let us take an example from the previous tutorial chapter:

In [ ]:
from proveit._common_ import A, B, C
from proveit.logic import Implies
%begin relabeling

In [ ]:
A_impl_B = Implies(A, B)

In [ ]:
B_from_A = A_impl_B.derive_consequent(assumptions={A, A_impl_B})

Now we will attempt to perform a relabeling derivation step by calling the `relabel` method on a **Judgment**.  This will not work because we are not allowed to relabel a **Variable** in the list of assumptions.  Otherwise we would end up being able to prove false statements (unless we relabeled the assumptions in a consistent manner, but **Prove-It** has a different mechanism for doing this as we shall see next).

In [ ]:
from proveit import RelabelingFailure
try:
    B_from_A.relabel({B:C})
    assert False, "Expecting an RelabelingFailure error; should not make it to this point"
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

## Basic relabeling

If we convert the assumptions to explicit hypotheses first, then we can do the relabeling that we failed to do before.

In [ ]:
explicit_B_from_A = B_from_A.as_implication(A_impl_B).as_implication(A)

This is a true statement that requires no assumptions.  This statement is valid for any value of $A$ and $B$ according to the rule that an implication is true as long as the conclusion is true whenever the hypothesis is true (but otherwise being indifferent to truth-aptness).  Now we will relabel $B$ to $C$ by calling **Judgment**'s `relabel` method.

In [ ]:
explicit_C_from_A = explicit_B_from_A.relabel({B:C})

Here is the full proof:

In [ ]:
explicit_C_from_A.proof()

Note, however, that a **Variable** may only be relabeled to another **Variable**.  To do anything else, *instantiation* would be required.  We will discuss *instantiation* in the next tutorial chapter.

In [ ]:
from proveit.logic import And
try:
    explicit_B_from_A.relabel({B:And(B, C)})
    assert False, "Expecting an RelabelingFailure error; should not make it to this point"
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

## Simultaneous relabeling

Relabeling will occur simultaneously in a consistent manner.  For example, we can switch labels.

In [ ]:
explicit_B_from_A.relabel({A:B, B:A})

Let us try a case with three labels that we will cycle.

In [ ]:
nested_impl = Implies(A, Implies(B, C))

In [ ]:
CfromNestedImpl = nested_impl.derive_consequent([nested_impl, A]).derive_consequent([nested_impl, A, B])

In [ ]:
cascading_impl = CfromNestedImpl.as_implication(nested_impl).as_implication(B).as_implication(A)

Convincing yourself that this is correct is left as an exercise to the reader (you could make a truth table and/or check the logic that got us to this point).  In any case, now we will demonstrate cyclic relabeling.

In [ ]:
relabeled_cascading_impl = cascading_impl.relabel({A:B, B:C, C:A})

Let us look at this proof:

In [ ]:
relabeled_cascading_impl.proof()

## Duplicated relabeling

It is possible to relabel multiple **Variable**s to the same **Variable** to derive a **Judgment** that is weaker than the original statement.

In [ ]:
redundant_cascading_impl = cascading_impl.relabel({A:B})

In [ ]:
redundant_cascading_impl.proof()

## Literals cannot be relabeled

We saw that **Literals** cannot be relabeled in *expression relabeling* in the <a href="tutorial01_basic_expr.ipynb">basic expressions chapter</a>.  Let us revisit this and verify that we cannot perform a relabeling derivation step.

In [ ]:
from proveit import Literal
X, Y, Z = Literal('tutorial', 'X'), Literal('tutorial', 'Y'), Literal('tutorial', 'Z')

In [ ]:
XYZimpl = Implies(X, Implies(Y, Z))

Let us try to relabel this.  Our first mistake will be that the expression is not a **Judgment**.  **Expression** has a `relabeled` method, but to call `relabel` (and make an actual derivation step), we need to start with a **Judgment**.

In [ ]:
from proveit import RelabelingFailure
try:
    XYZimpl.relabel({Y:C})
except AttributeError as e:
    print("EXPECTED ERROR:", e)

Let us make this an axiom to make it a **Judgment** by fiat.  This is not normally how axioms are created, but we will get into that in a later tutorial chapter

In [ ]:
from proveit import Axiom, Theory
XYZimplAxiom = Axiom(XYZimpl, Theory(), 'XYZimplAxiom')

Now we will try to `relabel`:

In [ ]:
from proveit import RelabelingFailure
try:
    XYZimplAxiom.proven_truth.relabel({Y:C})
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

That time, we got the error message that only **Variable**s may be relabeled.

## Scoping violation

We also noted with *expression relabeling* in <a href="tutorial01_basic_expr.ipynb">basic expressions chapter</a> that we are not allowed to violate the scoping restrictions of **Lambda** expressions.  We revisit this for the case of the *relabeling* derivation step.

For our examples, we will invoke the infinite geometic series theorem from `proveit.numbers.summation`:

In [ ]:
from proveit.numbers.summation._theorems_ import inf_geom_sum

In [ ]:
inf_geom_sum

Let us look how this expression is deconstructed in the expression information.  Note the use of **Lambda** mappings for the `Forall` *instance expression* (entry 2 below) and the `Sum` *summand* (entry 12 below).  These **Lambda** mappings define new scopes for $x$ and $m$ respectively.  This concept of operating on a **lambda** map will be discussed more in the next chapter.

In [ ]:
inf_geom_sum.expr_info()

Now we will use relabeling on the geometric series theorem to prove that we can write it in terms of $a$ and $n$ instead of $x$ and $m$.

In [ ]:
from proveit._common_ import a, n, m, x
inf_geom_sum.relabel({x:a, m:n})

We can also swap the labels.

In [ ]:
inf_geom_sum.relabel({x:m, m:x})

In [ ]:
inf_geom_sum.relabel({x:m, m:x}).proof()

However, we are subject to **Lambda** scoping restrictions if we try to relabel in any manner such that they map to the same **Variable** (or the meaning could be changed in a manner that is not strictly weaker).

In [ ]:
from proveit import ScopingViolation
try:
    inf_geom_sum.relabel({m:x})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

In [ ]:
try:
    inf_geom_sum.relabeled({x:m})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

In [ ]:
try:
    inf_geom_sum.relabeled({x:n, m:n})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

In [ ]:
%end relabeling

# Next chapter: <a href="tutorial05_forall.ipynb">Universal Quantification (Forall)</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>